In [252]:
# Library Import
import os 
import pandas as pd
import importlib.util

if importlib.util.find_spec("pandas") is None:	
	os.system("pip install pandas")

pd.options.mode.use_inf_as_na = True

In [253]:
# Parameters

Host = "https://strategenics.atlassian.net/"
Username = "mike.burns@strategenics.com.au"
Password = "ATATT3xFfGF0wcJfZvitiiWKYW_9UCdV2r1dWJE6O8a7ZxFRQUU7FVnBkdYlxNx9PcEn_tbHM-tdxz4DZedCozWuNUSI46DcZDPMKpXq0QDgiWomXTEkzDjVZfVa4oRjADfcbgebgME5skwsc5Ue4doiLA1OpiIyxzP1gzXAzNcfYSniUnKKoqI=2FD01AC0"
ValidProjectCategories = ["'Customer Delivery Projects'"]

In [254]:
# Functions
import base64
import pandas as pd
import numpy as np
import re 
import requests
import json
from functools import reduce

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnSearch(jql, fields = None, expand = None):
    def ApiCall(startAt) :
        url = "/rest/api/latest/search"
        headers = fnGetDefaultHeaders()
        defaultContents = {
            "startAt": startAt,
            "maxResults": "2",
            "jql": jql
        }
        if fields is not None:
            defaultContents["fields"] = fields.tolist()
        if expand is not None and expand != "":
            defaultContents["expand"] = expand        

        display(json.dumps(defaultContents))

        response = requests.post(Host + url, headers = headers, data = defaultContents)
        return response.json()
    return fnAPI(ApiCall)  

def fnGetIssueTypeFields(IssueTypes):
    def ApiCall(startAt) :
        url = "rest/api/latest/issue/createmeta"
        headers = fnGetDefaultHeaders()
        params = {
            "expand": "projects.issuetypes.fields",
            "projectKeys": ','.join(fnGetValidProjectKeys()["Key"].values),
            "issuetypeNames": ','.join(IssueTypes).replace("'", "")
        }
        response = requests.get(Host + url, headers = headers, params = params)
        return response.json()
    
    df = fnAPI(ApiCall)
    df = df.drop(["Expand"], axis=1)
    df = df.explode("Projects")
    df = fnExpandColumn(df, "Projects", None)
    df = df[["Issuetypes"]]
    df = df.explode("Issuetypes")
    df = fnExpandColumn(df, "Issuetypes", None)
    df = df[["Fields"]]
    df = fnExpandColumn(df, "Fields", None)

    df = df.loc[:,~df.columns.duplicated()]

    values = []
    for x in df.columns:        
        try:            
            valid = pd.DataFrame( df[~df[x].isnull()] )[[x]].iloc[0].get(0)
            values.append({
                "fieldId": valid['key'],
                "name": valid['name'],
                "schema_type": valid['schema']['type'],
                "required": valid['required']
            })
        except:
            display(x)
            pass

    values.append({ "fieldId": 'status', "name": 'Status', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'created', "name": 'Created', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'updated', "name": 'Updated', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'resolution', "name": 'Resolution', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'resolutiondate', "name": 'Resolution Date', "schema_type": 'date', "required": False })
    values.append({ "fieldId": 'lastViewed', "name": 'Last Viewed', "schema_type": 'date', "required": True })

    df = pd.DataFrame(values)

    df = df.sort_values("fieldId")
    return df 

def fnGetValidProjectKeys():
    def ApiCall(startAt) :
        url = "/rest/api/latest/project"
        headers = fnGetDefaultHeaders()
        params = { }
        response = requests.get(Host + url, headers = headers, params = params)
        return response.json()
    df = fnAPI(ApiCall)
    df = fnExpandColumn(df, "Project Category")
    df = df.loc[df['Project Category Name'].isin(ValidProjectCategories) | ("'" + df['Project Category Name'] + "'").isin(ValidProjectCategories)]
    return df[["Key"]]

def fnSentenceCase(s):
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]).replace("0", "")

def fnAPI(webRequestDelegate, startAt = 0): 
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))         
        except:
            return list(reduce(lambda x, y: x + y, [frm], [])) 
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    df.columns = [fnSentenceCase(x) for x in df.columns]
    return df
    
def fnExpandColumn(df:pd.DataFrame, colName:str, prefix:str = "Prefix "):
    if prefix == "Prefix ":
        prefix = colName + " "
    if prefix == None: 
        df = pd.concat([df.drop(colName, axis=1), df[colName].apply(lambda x: pd.Series(x))], axis=1)
    else:
        df = pd.concat([df.drop(colName, axis=1), df[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))], axis=1)
    df.columns = [fnSentenceCase(x) for x in df.columns]
    return df

In [255]:
# Issues Epics

from IPython.display import display
import pandas as pd

ExpectedIssueTypes = [ "Epic" ]
JQL = "issuetype in (" + ','.join(ExpectedIssueTypes) + ") and category in (" + ','.join(ValidProjectCategories) + ") ORDER BY updatedDate DESC" 

fields = fnGetIssueTypeFields(ExpectedIssueTypes)

display(JQL)

display(','.join(fields["fieldId"].values))

df = fnSearch(JQL, fields["fieldId"].values)

display(df)

df = df.drop(["Expand", "Start At", "Max Results", "Total"], axis=1)
df = df.explode("Issues")
df = fnExpandColumn(df, "Issues", None)
df = df.drop(["Expand", "Self"], axis=1)
df = fnExpandColumn(df, "Fields", None)
df = df.convert_dtypes().infer_objects()
df["Id"] = df["Id"].astype('Int64')

display(df.dtypes)
display(df)

"issuetype in (Epic) and category in ('Customer Delivery Projects') ORDER BY updatedDate DESC"

'assignee,attachment,components,created,customfield_10004,customfield_10007,customfield_10008,customfield_10009,customfield_10102,customfield_10600,customfield_10601,customfield_11100,customfield_11400,customfield_11506,customfield_11617,customfield_11618,customfield_11619,customfield_11620,customfield_11711,description,duedate,environment,eu.softwareplant.bigpicture__mode,eu.softwareplant.bigpicture__risk-consequence,eu.softwareplant.bigpicture__risk-probability,fixVersions,io.tempo.jira__account,issuelinks,issuetype,labels,lastViewed,parent,priority,project,reporter,resolution,resolutiondate,status,summary,timetracking,updated,versions'

'{"startAt": 0, "maxResults": "2", "jql": "issuetype in (Epic) and category in (\'Customer Delivery Projects\') ORDER BY updatedDate DESC", "fields": ["assignee", "attachment", "components", "created", "customfield_10004", "customfield_10007", "customfield_10008", "customfield_10009", "customfield_10102", "customfield_10600", "customfield_10601", "customfield_11100", "customfield_11400", "customfield_11506", "customfield_11617", "customfield_11618", "customfield_11619", "customfield_11620", "customfield_11711", "description", "duedate", "environment", "eu.softwareplant.bigpicture__mode", "eu.softwareplant.bigpicture__risk-consequence", "eu.softwareplant.bigpicture__risk-probability", "fixVersions", "io.tempo.jira__account", "issuelinks", "issuetype", "labels", "lastViewed", "parent", "priority", "project", "reporter", "resolution", "resolutiondate", "status", "summary", "timetracking", "updated", "versions"]}'

,Error Messages
0,[There was an error parsing JSON. Check that y...


KeyError: "['Expand', 'Start At', 'Max Results', 'Total'] not found in axis"